# Modelo de Vasicek

## Ecuaciones del Modelo

El modelo de Vasicek para la tasa instantánea en la medida histórica es:

$$
dr_t=\gamma\left(\overline{r}-r_t\right)dt+\sigma dX_t
$$

Este modelo también supone que $\lambda$ el *market price of interest rate risk* es constante y por lo tanto, en la medida ajustada por riesgo:

$$
dr_t=\gamma^*\left(\overline{r}^*-r_t\right)dt+\sigma dX^*_t
$$

donde $\gamma^*=\gamma$ y $\overline{r}^*=\overline{r}-\lambda\sigma\gamma^{-1}$.

La solución de esta ecuación es:

$$
r_t=r_0 e^{-\gamma^* t}+\overline{r}^*\left(1-e^{-\gamma^* t}\right)+\sigma e^{-\gamma^* t}\int_{0}^{t}e^{\gamma^* s}dX_s
$$

Y además tenemos que:

$$
\mathbb{E}_s\left[r_t\right]=r_0 e^{-\gamma^* \left(t-s\right)}+\overline{r}^*\left(1-e^{-\gamma^* (t-s)}\right) \\
$$

$$
Var_s\left[r_t\right]=\frac{\sigma^2}{2\gamma^*}\left(1-e^{-2\gamma^* (t-s)}\right)
$$

El modelo de Vasick admite una fórmula explícita para el valor de un bono cupón cero que vence en $T$:

$$
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right] \\
Z\left(r_T,T,T\right)=1
$$

Donde:

$$
B\left(t,T\right)=\frac{1}{\gamma^*}\left\{1-\exp\left[-\gamma^*\left(T-t\right)\right]\right\}
$$

$$
A\left(t,T\right)=\left[B\left(t,T\right)-\left(T-t\right)\right]\left[\overline{r}^*-\frac{\sigma^2}{2{\gamma^*}^2}\right]-\frac{\sigma^2{B\left(t,T\right)}^2}{4\gamma^*}
$$

### Ejercicio

¿Qué distribución tiene el bono cupón cero $Z$ en el modelo de Vasicek?

In [ ]:
pass

## Simulación del Modelo

Vamos a trabajar primero con parámetros inventados. Haremos una simulación. Luego usando la fórmula para el bono cupón cero estudiaremos el ajuste del modelo a un set de datos reales.

In [39]:
from typing import List, Tuple
import scipy.optimize as opt
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [22]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

Damos de alta los parámetros del modelo:

In [3]:
gamma = 1
sigma = .01
r_ = .03
r0 = .009 # tasa corta a día de hoy

Veamos que forma tienen sus trayectorias:

In [17]:
def vasicek_path(r0: float,
                 gamma: float, r_: float, sigma: float,
                 num_dias: int = 263,
                 dias_agno: int = 264,
                 pasos_dia: int = 1) -> List[Tuple[float, float]]:
    """
    Retorna un camino de simulación del modelo de Vasicek.
    
    params:
    
    - r0: tasa inicial (t = 0) de la simulación
    - Parámetros del modelo:
      - gamma: velocidad de reversión
      - r_: tasa de largo plazo
      - sigma: volatilidad
    - num_dias: número de días en la trayectoria, incluyendo el instante t = 0
    - dias_agno: número de días hábiles por año
    - pasos_dia: número de pasos de simulación en 1 día
    
    return:
    
    - Un `list` donde cada elemento es una `tuple` con los valores del tiempo y la tasa simulada.
    """
    dt = 1 / (dias_agno * pasos_dia)
    dt_gamma_r_ = dt * gamma * r_
    sigma_sqdt = sigma * math.sqrt(dt)
    result = [(0, r0),]
    
    r = r0
    for i in range(1, (num_dias + 1) * pasos_dia):
        r = dt_gamma_r_ + (1 - dt) * r + sigma_sqdt * np.random.normal()
        result.append((i * dt, r))
    return result

In [18]:
sim = vasicek_path(r0, gamma, r_, sigma)
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim, x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek. r0={r0:.2%}, gamma={gamma}, r_={r_:.2%}, sigma={sigma:.2%}'
)
fig.show()

## Estimación de Parámetros

Se utilizará la curva OIS que se construyó en clase (notebook 5):

In [19]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [23]:
df_curva.head().style.format(frmt)

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


In [20]:
# zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

In [57]:
fig = px.line(df_curva, x='plazo', y='tasa', title='Curva de Mercado')
fig.show()

Se definen las funciones asociadas a la fórmula para obtener un factor de descuento o precio de un bono cero cupón con el modelo de Vasicek.

In [114]:
def b_vasicek(r: float,
              r_: float,
              gamma: float,
              t: float,
              T: float) -> float:
    """
    Implementa la función B que interviene en el cálculo del valor de un bono cupón
    cero con el modelo de Vasicek.
    
    params:
    
    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)
    
    return:
    
    - valor de la función B
    """
    return 1 / gamma * (1 - math.exp(-gamma * (T - t)))

def a_vasicek(r: float,
              r_: float,
              gamma: float,
              sigma: float,
              t: float,
              T: float) -> float:
    """
    Implementa la función A que interviene en el cálculo del valor de un bono cupón
    cero con el modelo de Vasicek.
    
    params:
    
    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)
    
    return:
    
    - valor de la función A
    """
    b = b_vasicek(r, r_, gamma, t, T)
    sigma2 = sigma ** 2
    return (b - (T - t)) * (r_ - sigma2 / (2 * gamma ** 2)) - (sigma2 * b ** 2) / (4.0 * gamma)

def zero_vasicek(r: float,
                 r_: float,
                 gamma: float,
                 sigma: float,
                 t: float,
                 T: float) -> float:
    """
    Implementa la fórmula para el cálculo del valor de un bono cupón cero con el modelo de Vasicek.
    
    params:
    
    - r: valor de la tasa corta
    - Parámetros del Modelo:
      - r_: tasa de largo plazo
      - gamma: velocidad de reversión
    - t: tiempo de cálculo
    - T: tiempo de vencimiento del bono (debe ser T >= t)
    
    return:
    
    - valor de la función A
    """
    return math.exp(a_vasicek(r, r_, gamma, sigma, t, T) - b_vasicek(r, r_, gamma, t, T) * r)

In [115]:
# Probar la función
df = zero_vasicek(r0, r_, gamma, sigma, 0, 1.5)
print(df)

0.9717422690073619


Queremos estimar los parámetros $\gamma^*$, $\overline{r}^*$ y $\sigma$ minimizando el error cuadrático medio entre los factores de descuento obtenidos con la fórmula del modelo y los factores de descuento de la curva de mercado (en `df_curva` más arriba).

In [116]:
for row in df_curva.itertuples():
    print(f'{row.plazo}, {row.tasa:.4%}, {row.df:.6%}')

1, 0.0811%, 99.999778%
7, 0.0841%, 99.998388%
14, 0.0780%, 99.997010%
21, 0.0774%, 99.995549%
33, 0.0781%, 99.992942%
61, 0.0781%, 99.986954%
92, 0.0811%, 99.979560%
125, 0.0781%, 99.973271%
152, 0.0760%, 99.968343%
182, 0.0750%, 99.962603%
212, 0.0736%, 99.957265%
243, 0.0725%, 99.951761%
273, 0.0720%, 99.946187%
306, 0.0714%, 99.940196%
335, 0.0708%, 99.934996%
365, 0.0702%, 99.929787%
547, 0.0642%, 99.903893%
730, 0.0568%, 99.886549%
1097, 0.0750%, 99.774813%
1462, 0.1258%, 99.497569%
1826, 0.1939%, 99.034837%
2191, 0.2775%, 98.348253%
2556, 0.3603%, 97.508202%
2924, 0.4384%, 96.548853%
3288, 0.5117%, 95.495266%
3653, 0.5812%, 94.349488%
4383, 0.6917%, 92.028980%
5479, 0.8064%, 88.599741%
7306, 0.9253%, 83.092401%
9133, 0.9657%, 78.533775%
10957, 0.9881%, 74.332619%
14610, 0.9408%, 68.619685%
18262, 0.8464%, 65.475678%


Se define la función objetivo para encontrar los parámetros:

In [97]:
# x = [r_, gamma, sigma] valor inicial
def objective(x: List[float], *args) -> float:
    """
    Calcula el error cuadrático total entre los valores de la curva de mercado y los obtenidos
    con la fórmula del modelo de Vasicek y los parámetros x.
    """
    error = 0.0
    for row in args[0].itertuples():
        error += (row.df - zero_vasicek(args[1], x[0], x[1], x[2], 0, row.plazo / 365)) ** 2
    return error

Se encuentran los parámetros usando scipy:

In [130]:
x0 = [.03, 0.5, .01]
r0 = df_curva.iloc[0]['tasa']
bnds = ((0, None), (0, None), (0, None))
result = opt.minimize(objective, x0, args=(df_curva, r0), bounds=bnds, tol=.00000001)
print(result)
optimo = result.x

      fun: 0.003306986381573132
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([6.94039444e-05, 6.28577052e-07, 5.20417043e-09])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 216
      nit: 38
     njev: 54
   status: 0
  success: True
        x: array([0.01031726, 0.22319265, 0.        ])


In [131]:
print(f'r_: {optimo[0]:.4%}')
print(f'gamma: {optimo[1]:.4f}')
print(f'sigma: {optimo[2]:.4%}')

r_: 1.0317%
gamma: 0.2232
sigma: 0.0000%


Comparemos la curva real con la aproximación:

In [132]:
v_curva = []
for row in df_curva.itertuples():
    v_curva.append((
        row.plazo / 365.0,
        row.df,
        zero_vasicek(r0, optimo[0], optimo[1], optimo[2], 0, row.plazo / 365)
    ))
compara_dfs = pd.DataFrame(v_curva, columns=['plazo', 'df_mkt', 'df_vasicek'])
fig = px.line(compara_dfs, x='plazo', y=['df_mkt', 'df_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

Hagamos la misma comparación, pero en tasa:

In [133]:
def tasa_comp(df: float, plazo: float) -> float:
    return (1 / df) ** (1 / plazo) - 1

compara_dfs['tasa_vasicek'] = compara_dfs.apply(lambda row: tasa_comp(row.df_vasicek, row.plazo), axis=1)
compara_dfs['tasa_mkt'] = compara_dfs.apply(lambda row: tasa_comp(row.df_mkt, row.plazo), axis=1)

In [134]:
compara_dfs.head()

,plazo,df_mkt,df_vasicek,tasa_vasicek,tasa_mkt
0,0.002740,0.999998,0.999998,0.000814,0.000811
1,0.019178,0.999984,0.999984,0.000832,0.000841
2,0.038356,0.999970,0.999967,0.000852,0.000780
3,0.057534,0.999955,0.999950,0.000872,0.000774
4,0.090411,0.999929,0.999918,0.000907,0.000781


In [135]:
fig = px.line(compara_dfs, x='plazo', y=['tasa_mkt', 'tasa_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

Veamos como queda la simulación:

In [136]:
sim = vasicek_path(r0, optimo[0], optimo[1], optimo[2])
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])
fig = px.line(
    df_sim, x='t',
    y='tasa',
    title=f'Simulación Modelo Vasicek. r0={r0:.2%}, gamma={optimo[1]:.4f}, \
    r_={optimo[0]:.2%}, sigma={optimo[2]:.2%}'
)
fig.show()

In [137]:
# x = [r_, gamma] valor inicial
def objective2(x: List[float], *args) -> float:
    """
    Calcula el error cuadrático total entre los valores de la curva de mercado y los obtenidos
    con la fórmula del modelo de Vasicek y los parámetros x.
    """
    error = 0.0
    for row in args[0].itertuples():
        error += (row.df - zero_vasicek(args[1], x[0], x[1], args[2], 0, row.plazo / 365)) ** 2
    return error

In [138]:
x00 = [.03, 1]
sigma0 = .02
bnds = ((0, None), (0, None))
result = opt.minimize(objective2, x00, args=(df_curva, r0, sigma0), bounds=bnds)
print(result)
optimo2 = result.x

      fun: 0.0034946069945208007
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.74033964e-05, -3.93222781e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 219
      nit: 55
     njev: 73
   status: 0
  success: True
        x: array([0.02472949, 0.11358796])


In [139]:
print(f'r_: {optimo[0]:.4%}')
print(f'gamma: {optimo[1]:.4f}')
print(f'sigma: {sigma0:.4%}')

r_: 1.0317%
gamma: 0.2232
sigma: 2.0000%


In [140]:
v_curva = []
for row in df_curva.itertuples():
    v_curva.append((
        row.plazo / 365.0,
        row.df,
        zero_vasicek(r0, optimo2[0], optimo2[1], sigma0, 0, row.plazo / 365)
    ))
compara_dfs_2 = pd.DataFrame(v_curva, columns=['plazo', 'df_mkt', 'df_vasicek'])
fig = px.line(compara_dfs_2, x='plazo', y=['df_mkt', 'df_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()

In [141]:
compara_dfs_2['tasa_vasicek'] = compara_dfs_2.apply(lambda row: tasa_comp(row.df_vasicek, row.plazo), axis=1)
compara_dfs_2['tasa_mkt'] = compara_dfs_2.apply(lambda row: tasa_comp(row.df_mkt, row.plazo), axis=1)

In [142]:
fig = px.line(compara_dfs_2, x='plazo', y=['tasa_mkt', 'tasa_vasicek'], title='Compara Vasicek vs Mercado')
fig.show()